# Initialize

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 5.4 MB/s 


In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
data = pd.read_csv("/content/gdrive/My Drive/skripsi/covidtweets_newerer.csv")
print(data)

                                                 tweets  label
0     WALAU TELAH DIVAKSIN Protokol Kesehatan Tetap ...      4
1                        Ayoo Sukseskan Vaksin Covid-19      4
2               Agar Kita Sehat dan Selamat Ayo Vaksin.      4
3     Dalam penanganan Covid-19 pemerintah terus ber...      3
4     Walaupun sudah divaksin, Sedulur harus tetap p...      4
...                                                 ...    ...
2863  Berdasarkan pengalaman saya, efek samping vaks...      2
2864  Ini ada lagi berita palsu yang bilang air kela...      2
2865  Efek pasca vaksin memang berbeda beda, saya me...      2
2866  Sudah divaksin bukan berarti kebal, masih bisa...      2
2867    Gue gak mendukung atau menolak vaksin nusantara      2

[2868 rows x 2 columns]


# Casefolding

In [ ]:
tweet = data['tweets']
pp1 = tweet.str.lower()
print(pp1)

0       walau telah divaksin protokol kesehatan tetap ...
1                          ayoo sukseskan vaksin covid-19
2                 agar kita sehat dan selamat ayo vaksin.
3       dalam penanganan covid-19 pemerintah terus ber...
4       walaupun sudah divaksin, sedulur harus tetap p...
                              ...                        
2863    berdasarkan pengalaman saya, efek samping vaks...
2864    ini ada lagi berita palsu yang bilang air kela...
2865    efek pasca vaksin memang berbeda beda, saya me...
2866    sudah divaksin bukan berarti kebal, masih bisa...
2867      gue gak mendukung atau menolak vaksin nusantara
Name: tweets, Length: 2868, dtype: object


# Special Word Removal

In [ ]:
wr1= pp1.str.replace('URL', '')
wr2 = wr1.str.replace('SENSITIVE-NO','')
pp2 = wr2.str.replace("[^a-zA-Z0-9]+"," ") #non alphanumeric

print(pp2)

0       walau telah divaksin protokol kesehatan tetap ...
1                          ayoo sukseskan vaksin covid 19
2                 agar kita sehat dan selamat ayo vaksin 
3       dalam penanganan covid 19 pemerintah terus ber...
4       walaupun sudah divaksin sedulur harus tetap pa...
                              ...                        
2863    berdasarkan pengalaman saya efek samping vaksi...
2864    ini ada lagi berita palsu yang bilang air kela...
2865    efek pasca vaksin memang berbeda beda saya mer...
2866    sudah divaksin bukan berarti kebal masih bisa ...
2867      gue gak mendukung atau menolak vaksin nusantara
Name: tweets, Length: 2868, dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


# Stop Word Removal

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
stop = stopword.remove

pp3 = pp2.apply(stop)
print(pp3[2860:])

2860    indonesia gak umroh bukan vaksin sinovac karen...
2861           bukankah vaksin sinovac astrazeneca impor 
2862    media sosial viral komentar netizen mengenai k...
2863    berdasarkan pengalaman efek samping vaksin ber...
2864    ada berita palsu bilang air kelapa menetralkan...
2865    efek pasca vaksin memang berbeda beda merasaka...
2866      divaksin bukan berarti kebal bisa terkena covid
2867           gue gak mendukung menolak vaksin nusantara
Name: tweets, dtype: object


# Normalization

In [ ]:
singkatan = pd.read_csv("/content/gdrive/MyDrive/skripsi/kamus_singkatan.txt",sep=';')
singkatan.columns = ['non','norm']
print(singkatan)

          non      norm
0        adek     adik 
1        adlh   adalah 
2         aer      air 
3     aiskrim  es krim 
4          aj     saja 
...       ...       ...
1303      yoi       iya
1304      yup       ya 
1305     yups        ya
1306     yupz        ya
1307     ywdh  ya sudah

[1308 rows x 2 columns]


In [ ]:
!pip install flashtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9309 sha256=338ed0c4fb8d39e0fa94714418ab346a536a073241e03245977a089a785c76c5
  Stored in directory: /root/.cache/pip/wheels/cb/19/58/4e8fdd0009a7f89dbce3c18fff2e0d0fa201d5cdfd16f113b7
Successfully built flashtext


In [ ]:
change = dict(zip(singkatan.non,singkatan.norm))

from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor(case_sensitive=True)
for k, v in change.items():
    keyword_processor.add_keyword(k, v)
    
pp4 = pd.DataFrame(list(map(keyword_processor.replace_keywords, pp2)))
pp4.columns = ['tweets']
print(pp4[2860:])

                                                 tweets
2860  indonesia tidak bisa umroh bukan karena vaksin...
2861     bukankah vaksin sinovac dan astrazeneca impor 
2862  di media sosial viral komentar para netizen me...
2863  berdasarkan pengalaman saya efek samping vaksi...
2864  ini ada lagi berita palsu yang bilang air kela...
2865  efek pasca vaksin memang berbeda beda saya mer...
2866  sudah divaksin bukan berarti kebal masih bisa ...
2867  saya tidak mendukung atau menolak vaksin nusan...


In [ ]:
#check
print(len(keyword_processor))
keyword_processor['ga']

1242


'tidak'

In [ ]:
#download no stopword no stemming
#pp4.to_csv('/content/gdrive/MyDrive/skripsi/preprocessed/no_stopword_no_stemmed.csv')

# Download No Stemming

In [ ]:
#pp4.to_csv('/content/gdrive/MyDrive/skripsi/preprocessed/no_stemmed.csv')

# Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
pp5 = pp4['tweets'].apply(stemmer.stem)
print(pp5)

0       walau telah vaksin protokol sehat tetap jadi s...
1                             ayoo sukses vaksin covid 19
2                  agar kita sehat dan selamat ayo vaksin
3       dalam tangan covid 19 perintah terus koordinas...
4       walaupun sudah vaksin dulur harus tetap patuh ...
                              ...                        
2863    dasar alam saya efek samping vaksin gantung ke...
2864    ini ada lagi berita palsu yang bilang air kela...
2865    efek pasca vaksin memang beda beda saya rasa l...
2866    sudah vaksin bukan arti kebal masih bisa kena ...
2867        saya tidak dukung atau tolak vaksin nusantara
Name: tweets, Length: 2868, dtype: object


# Download After Stemming

In [ ]:
pp5.to_csv('/content/gdrive/MyDrive/skripsi/preprocessed_newerer/after_preprocessed.csv')

# Split No Stemmed Dataset

In [ ]:
df1 = pd.read_csv("/content/gdrive/My Drive/skripsi/preprocessed/no_stemmed/no_stemmed_labeled.csv")

In [ ]:
train, validate, test = \
              np.split(df1.sample(frac=1, random_state=42), 
                       [int(.75*len(df1)), int(.9*len(df1))])

In [ ]:
train

,tweets,label
1868,jangan mau kalah takut varian covid b117 tetap...,4
1272,who kasus pembekuan darah astrazeneca sangat j...,3
571,kabar corona dunia uji coba pfizer anak hingga...,3
941,seluruh peserta telah melaksanakan vaksinasi p...,4
1652,vaksinasi covid 19 lanjut bulan ramadhan orang...,2
...,...,...
491,target vaksin tetap akhir tahun presiden joko ...,4
953,vaksin teruji ampuh menghadapi covid kita duku...,4
1097,masyarakat benar meria usah takut ragu divaks...,4
1032,jangan embargo vaksin,4


In [ ]:
train.to_csv('/content/gdrive/MyDrive/skripsi/preprocessed/no_stemmed/data_train.csv')

In [ ]:
validate

,tweets,label
750,sebanyak 11 630 orang divaksin kesempatan sebe...,3
657,ayo vaksinasi biar cepat stop pandemi,4
234,vaksinasi masal terbukti menurunkan angka penu...,4
732,divaksin tetap jaga 5m,4
2095,tidak butuh vaksin pak butuh a ketenangan kete...,0
...,...,...
1662,doain saja semoga cepat berjalan program vak...,4
961,melawan covid butuh kekompakan kita dukung vak...,4
853,step 2 done lengkap,3
1311,efek embargo menkes budi sebut 100 juta vaksin...,1


In [ ]:
validate.to_csv('/content/gdrive/MyDrive/skripsi/preprocessed/no_stemmed/data_val.csv')

In [ ]:
test

,tweets,label
685,vaksin covid19 membuat kebal covid 19,2
1687,terimakasih pemerintah indonesia sudah divaksi...,4
569,peningkatan vaksin covid 19,3
565,idi banten ilmah vaksin astrazeneca ditemukan ...,0
17,selamatkah vaksin covid 19 ibu mengandung,2
...,...,...
1638,bpom banyak pengabaian tahapan uji klinis i va...,1
1095,vaksinasi lansia abaikan protokol kesehatan,1
1130,terkait pasokan vaksin menkes mengungkapkan pi...,3
1294,ppkm mikro vaksinasi 5m kunci penanganan covid 19,3


In [ ]:
test.to_csv('/content/gdrive/MyDrive/skripsi/preprocessed/no_stemmed/data_test.csv')

# Split After Stemmed Dataset

In [ ]:
df2 = pd.read_csv("/content/gdrive/My Drive/skripsi/preprocessed_newerer/after_preprocessed_newerer.csv")

In [ ]:
train, validate, test = \
              np.split(df2.sample(frac=1, random_state=42), 
                       [int(.75*len(df2)), int(.9*len(df2))])

In [ ]:
train

,tweets,label
443,presiden joko widodo jokowi tinjau vaksinasi m...,3
2394,ini jelas untuk anda yang sudah di vaksin tapi...,2
761,masalah benar bukan pada vaksinasi lain bagaim...,3
651,ayo vaksin dan terap prokes dengan 5m,4
2673,astrazeneca masalah tetap pakai oleh australia,1
...,...,...
1489,vaksinasi covid 19 dosis 1 untuk nakes lansia ...,3
2246,mama juga tidak berani vaksin,0
60,sementara untuk lansia terima vaksin dosis per...,3
2630,orang guru di jombang berontak tidak mau sunti...,0


In [ ]:
train.to_csv('/content/gdrive/MyDrive/skripsi/preprocessed_newerer/data_train9010.csv')

In [ ]:
validate

,tweets,label
92,vaksin bukan hanya untuk penting pribadi lain ...,4
666,vaksinasi bisa cegah virus covid 19,4
1887,india terlalu cepat puas diri vaksinasi massal...,1
280,pakar farmasi kini kembang vaksin covid 19 gen...,2
1719,tim kembang vaksin covid 19 nusantara minta un...,2
...,...,...
954,lawan covid butuh kompak mari kita dukung vaks...,4
2846,hingga saat ini ada lima jenis vaksin yang gun...,2
639,vaksin ada untuk sehat indonesia oleh karena m...,4
2659,akal akal orang kaya saja ini bikin pandemi bi...,0


In [ ]:
validate.to_csv('/content/gdrive/MyDrive/skripsi/preprocessed_newerer/data_val9010.csv')

In [ ]:
test

,tweets,label
1158,vaksinasi massal layan publik,3
1415,bpom terbit vaksin,4
2594,daftar lansia di sekitar untuk segera vaksin c...,4
1527,mufti dari al azhar mesir dan saudi kompak nya...,3
2795,apa erti vaksin,2
...,...,...
1638,sebab mati lansia usai vaksin covid 19 di jepa...,1
1095,rekomendasi vaksinasi dari who lama pandemi co...,3
1130,mulai 1 ramadhan saudi izin jamaah umroh yang ...,3
1294,mnc trijaya fm saji spesial program kita yakin...,3


In [ ]:
test.to_csv('/content/gdrive/MyDrive/skripsi/preprocessed_newerer/data_test9010.csv')